# 1. Contextualização e Entendimento dos Dados

**Objetivo:** Analisar dados históricos de solicitações de crédito para identificar padrões de risco e inadimplência.
<br>
**Contexto:** O objetivo final é analisar a correlação entre a variável `TARGET` (0 = Pagador, 1 = Inadimplente) e as variáveis auxiliares a fim de identificar padrões e pré diagnosticar possíveis bons ou mal pagadores para minimizar perdas financeiras.
<br>
**Metodologia:**: Utilizaremos a etapa de *Data Understanding* do **CRISP-DM** para coletar, entender e selecionar os dados mais relevates para a análise

# 2. Configuração do Ambiente

Nesta seção serão apresentadas as configurações utilizadas para esta etapa de `Data Understanding`

## 2.1 Importação de Bibliotecas

In [2]:
import pandas as pd

# 3. Coleta dos Dados (Data Collection)

Todos os dados utiizados nesse projeto estão disponíveis publicamente na plataforma **Kaggle**.

# 3.1 Coleta dos Dados

Os dados foram obtidos via API do Kaggle e foram armazenados localmente em `data/`. No total, foram importados 10 arquivos `.csv`. Link para repositório do Kaglle: https://www.kaggle.com/datasets/megancrenshaw/home-credit-default-risk 

Inicialmente utilizaremos os dados de `HomeCredit_columns_description.csv` para entender mais sobre os nossos dados e selecionar os que podem ser mais relevantes. Ao final escolheremos os dados utilizados e criaremos algumas hipóteses que serão respondidas no final dos Notebooks

# 4. Definição de domínio do Problema e Seleção de Variáveis

Para esta fase do projeto, utilizaremos apenas uma tabela, a `applcation_train.csv` pois somente com ela já podemos obter insights valiosos e ela nos permite a análise de variáveis que podem interferir na inadimplência de clientes.
Utilizaremos também um dicionário de dados disponivel em `data/HomeCredit_columns_description.csv` para analisar as descrições de cada variável para selecionar as que forem mais relevantes.

## 4.1 Seleção de Variáveis e Dicionário de Dados

In [3]:
#Loading the description file
description_df = pd.read_csv('../data/HomeCredit_columns_description.csv', sep=',')
description_df.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


Agora, iremos separar apenas as variáveis contidas em `application_train`

In [4]:
# Creating a function to filter descriptions for specific tables
selected_table = 'application_{train|test}.csv'

def filter_description(df):

    return (df[df['Table'] == selected_table]
            .drop(columns=['Table', 'Unnamed: 0']) #Removing unnecessary columns
            )

In [5]:
description_list = filter_description(description_df)
description_list

,Row,Description,Special
0,SK_ID_CURR,ID of loan in our sample,NaN
1,TARGET,Target variable (1 - client with payment diffi...,NaN
2,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,CODE_GENDER,Gender of the client,NaN
4,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...
117,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN
118,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN
119,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN
120,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN


Aqui, visando aplicação dos conheicmentos e pensando em estratégias de negócio escolheremos as colunas pensando nos 5 C's do Crédito: (Character, Capacity, Capital, Collateral, Conditions)

1. **Variáveis Essenciais**
    - `TARGET`: A variável alvo (0 = Todos os outros casos, 1 = Dificuldades com o pagamento)
    - `SK_ID_CURR`: Identificador único do empréstimo
    - `NAME_CONTRACT_TYPE`: Tipo de empréstimo

2. **Character**: Aqui analisaremos a saúde financeira do cliente
    - `AMT_INCOME_TOTAL`: Renda total
    - `AMT_CREDIT`: Valor do empréstimo pedido
    - `AMT_ANNUITY`: Valor da parcela anual
    - `AMT_GOODS_PRICE`: Preço do Bem (Caso seja um financiamento de bens específicos)

3. **Perfil do Cliente**
    - `CODE_GENDER`: Gênero do cliente
    - `DAYS_BIRTH`: Idade em dias
    - `NAME_EDUCATION_TYPE`: Maior nível de escolaridade (É uma variável "Ordinal" (Ensino Médio < Superior < Pós). Você pode explorar Label Encoding aqui.)
    - `NAME_FAMILY_STATUS`: Status Civil
    - `CNT_CHILDREN`: Quantidade de filhos
    - `REGION_RATING_CLIENT`: Região de moradia (1, 2 e 3)

4. **Estabilidade e Trabalho**
    - `DAYS_EMPLOYED`: Dias empregado (Pegadinha Famosa (Bônus para seu Portfólio): Essa coluna tem um valor "mágico" (365243) para desempregados/aposentados que distorce a média. Descobrir e tratar isso na sua EDA mostra nível Senior.)
    - `NAME_INCOME_TYPE`: Tipo de renda (Empresário, assalariado, pensionista, etc)
    - `OCCUPATION_TYPE`: Profissão

5. **Ativos e Score Externo**
    - `FLAG_OWN_CAR`: Flag para se o cliente possui um carro
    - `FLAG_OWN_REALTY`: Flag para se o cliente possui uma casa
    - `OWN_CAR_AGE`: Idade do carro
    - `EXT_SOURCE_1`: Score normalizado de outra agência de crédito
    - `EXT_SOURCE_2`: Score normalizado de outra agência de crédito
    - `EXT_SOURCE_3`: Score normalizado de outra agência de crédito


In [6]:
#Separating only selected columns
selected_columns = [
    # Identifier and Target
    'TARGET', 'SK_ID_CURR',

    # Characteristics of the loan
    'NAME_CONTRACT_TYPE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 
    'AMT_ANNUITY', 'AMT_GOODS_PRICE',

    # Demographic profile
    'CODE_GENDER', 'CNT_CHILDREN', 'DAYS_BIRTH', 
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',

    # Work and Stability
    'DAYS_EMPLOYED', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',

    # Location and Heritage
    'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'OWN_CAR_AGE',
    'REGION_RATING_CLIENT', 
    
    # External Score
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]

In [7]:
df = pd.read_csv('../data/application_train.csv', sep=',')
df = df[selected_columns]

# 5. Descrição dos dados (Data Description)

Aqui após a filtragem analisaremos algumas características nos dados selecionado

## 5.1 Análise Geral dos Dados

In [8]:
df.head()

,TARGET,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,CODE_GENDER,CNT_CHILDREN,DAYS_BIRTH,...,DAYS_EMPLOYED,NAME_INCOME_TYPE,OCCUPATION_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,OWN_CAR_AGE,REGION_RATING_CLIENT,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,1,100002,Cash loans,202500.0,406597.5,24700.5,351000.0,M,0,-9461,...,-637,Working,Laborers,N,Y,NaN,2,0.083037,0.262949,0.139376
1,0,100003,Cash loans,270000.0,1293502.5,35698.5,1129500.0,F,0,-16765,...,-1188,State servant,Core staff,N,N,NaN,1,0.311267,0.622246,NaN
2,0,100004,Revolving loans,67500.0,135000.0,6750.0,135000.0,M,0,-19046,...,-225,Working,Laborers,Y,Y,26.0,2,NaN,0.555912,0.729567
3,0,100006,Cash loans,135000.0,312682.5,29686.5,297000.0,F,0,-19005,...,-3039,Working,Laborers,N,Y,NaN,2,NaN,0.650442,NaN
4,0,100007,Cash loans,121500.0,513000.0,21865.5,513000.0,M,0,-19932,...,-3038,Working,Core staff,N,Y,NaN,2,NaN,0.322738,NaN


## 5.1 Descrição Estatística e Tipos de Dados

Agora analisaremos estatísticas básicas sobre os dados, volumetria e tipo de dados 

### 5.1.1 Tamanho do Dataset

In [9]:
print(f"Lines: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")

Lines: 307511
Columns: 22


### 5.1.2 Tipos de Dados

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   TARGET                307511 non-null  int64  
 1   SK_ID_CURR            307511 non-null  int64  
 2   NAME_CONTRACT_TYPE    307511 non-null  object 
 3   AMT_INCOME_TOTAL      307511 non-null  float64
 4   AMT_CREDIT            307511 non-null  float64
 5   AMT_ANNUITY           307499 non-null  float64
 6   AMT_GOODS_PRICE       307233 non-null  float64
 7   CODE_GENDER           307511 non-null  object 
 8   CNT_CHILDREN          307511 non-null  int64  
 9   DAYS_BIRTH            307511 non-null  int64  
 10  NAME_EDUCATION_TYPE   307511 non-null  object 
 11  NAME_FAMILY_STATUS    307511 non-null  object 
 12  DAYS_EMPLOYED         307511 non-null  int64  
 13  NAME_INCOME_TYPE      307511 non-null  object 
 14  OCCUPATION_TYPE       211120 non-null  object 
 15  

### 5.1.3 Estatísticas Básicas (Variáveis Numéricas)

In [11]:
df.describe()

,TARGET,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,CNT_CHILDREN,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,REGION_RATING_CLIENT,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
count,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,134133.000000,3.068510e+05,246546.000000
mean,0.080729,278180.518577,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.417052,-16036.995067,63815.045904,12.061091,2.052463,0.502130,5.143927e-01,0.510853
std,0.272419,102790.175348,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.722121,4363.988632,141275.766519,11.944812,0.509034,0.211062,1.910602e-01,0.194844
min,0.000000,100002.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000000,-25229.000000,-17912.000000,0.000000,1.000000,0.014568,8.173617e-08,0.000527
25%,0.000000,189145.500000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.000000,-19682.000000,-2760.000000,5.000000,2.000000,0.334007,3.924574e-01,0.370650
50%,0.000000,278202.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.000000,-15750.000000,-1213.000000,9.000000,2.000000,0.505998,5.659614e-01,0.535276
75%,0.000000,367142.500000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,1.000000,-12413.000000,-289.000000,15.000000,2.000000,0.675053,6.636171e-01,0.669057
max,1.000000,456255.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,19.000000,-7489.000000,365243.000000,91.000000,3.000000,0.962693,8.549997e-01,0.896010


### 5.1.4 Valores possíveis para cada coluna (Não Numéricas)

In [28]:
#Columns of type object
columns = df.dtypes[df.dtypes == 'object'].index.tolist()

for col in columns:
    print(f"-----------{col}-----------")
    print(f"{df[col].unique()}")


-----------NAME_CONTRACT_TYPE-----------
['Cash loans' 'Revolving loans']
-----------CODE_GENDER-----------
['M' 'F' 'XNA']
-----------NAME_EDUCATION_TYPE-----------
['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
-----------NAME_FAMILY_STATUS-----------
['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']
-----------NAME_INCOME_TYPE-----------
['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
-----------OCCUPATION_TYPE-----------
['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']
-----------FLAG_OWN_CAR-----------
['N' 'Y']
-----------FLAG_OWN_REALTY-----------
['Y' 'N']


# 6. Verificação de qualidade inicial (Data Quality)

Aqui, segundo a metodologia CRSIP-DM teremos que ja apontar possíveis problemas no início. Para isso analisaremos valores nulos e verificação de duplicatas

## 6.1 Contagem de Valores Nulos

In [29]:
df.isnull().sum()

TARGET                       0
SK_ID_CURR                   0
NAME_CONTRACT_TYPE           0
AMT_INCOME_TOTAL             0
AMT_CREDIT                   0
AMT_ANNUITY                 12
AMT_GOODS_PRICE            278
CODE_GENDER                  0
CNT_CHILDREN                 0
DAYS_BIRTH                   0
NAME_EDUCATION_TYPE          0
NAME_FAMILY_STATUS           0
DAYS_EMPLOYED                0
NAME_INCOME_TYPE             0
OCCUPATION_TYPE          96391
FLAG_OWN_CAR                 0
FLAG_OWN_REALTY              0
OWN_CAR_AGE             202929
REGION_RATING_CLIENT         0
EXT_SOURCE_1            173378
EXT_SOURCE_2               660
EXT_SOURCE_3             60965
dtype: int64

## 6.2 Verificação de Duplicatas

In [37]:
has_duplicates = df.duplicated().any()
print(has_duplicates)  # Returns True if duplicates exist

False


# 7. Conclusões

Após essa análise inicial dos dados podemos concluir que:
- Volume: Trabalharemos com um dataset de 307511 linhas por 22 colunas (8 -> float, 6 -> int, 8 -> object)
- A maioria do dataset possui clientes não mals pagadores
- As variáveis do tipo object estão bem definidas, será necessário apenas transformá-las em ordinais (sem necessidade de tratamento para valores inconsistentes)
- Foram detectados variáveis com muitos valores nulos (`OCCUPATION_TYPE`, `OWN_CAR_AGE`, `EXT_SORCE_{1|3}`) porém deveremos analisar se a ausência desses valores se da pela falta de dados ou outra causa, como por exemplo a não necessidade de preenchimento dessa informação (Ex: Se a pessoa não tem um emprego então a coluna de tipo de emprego não tem porque ser preenchida)
- Para tratar esse caso de valores nulos para valores numéricos, poderá ser substituido valor 0 ou então média.
- Para valores do tipo object, talvez será necessário criar uma classe nova

## 7.1 Próxima Etapa

A próxima etapa será a Preparaçao dos Dados (Data Preparation). Nela, realizaremos a limpeza dos valores nulos identificados, a conversão de tipos e a criação de novas variáveis (Feature Engineering) essenciais para a análise, como a transformação da idade em anos e proporção de renda.

